# Dask Distributed

In the previous notebook we used Dask.delayed to create computations.  By default, these ran on a local thread pool.  Often, this is sufficient, especially when you are bound by NumPy and Pandas routines which release the GIL and when you are using powerful workstation computers with many cores.

However sometimes you may want to execute your code in processes (for Pure Python code that holds onto the GIL), in a single thread (for profiling and debugging) or across a cluster (for larger computations).

In this section we will first talk about changing schedulers.  Then we'll use the dask.distributed scheduler in more depth.

### Previous exercise

One solution to the previous exercise follows below.  We run this same computation using three single-machine schedulers:

1.  dask.threaded.get
2.  dask.multiprocessing.get
3.  dask.async.get_sync

In each case we change the scheudler by providing a `get=` keyword argument like the following:

```python
total.compute(get=dask.multiprocessing.get)
# or 
dask.compute(a, b, get=dask.multiprocessing.get)
```

In [ ]:
import dask 
import dask.multiprocessing
import pandas as pd
from glob import glob
import os

In [ ]:
%%time

filenames = sorted(glob(os.path.join('data', 'stocks', 'GOOG', '*.csv')))

spreads = []
days = []
for fn in filenames:
    df = dask.delayed(pd.read_csv)(fn, parse_dates=['timestamp'], index_col='timestamp')
    spread = df.high.max() - df.low.min()
    day = df.index[0].date()
    
    spreads.append(spread)
    days.append(day)

In [ ]:
%time s, d = dask.compute(spreads, days)  # this uses threads by default

In [ ]:
%time s, d = dask.compute(spreads, days, get=dask.multiprocessing.get)  # this uses processes by default

In [ ]:
%time s, d = dask.compute(spreads, days, get=dask.async.get_sync)  # This uses a single thread by default

### Question:  In what cases would you want to use one scheduler over another?

http://dask.pydata.org/en/latest/scheduler-choice.html

### Distributed Scheduler

The dask.distributed system is composed of a single centralizd scheduler and several worker processes.  We can follow the instructions here to set up dask.distributed on our local computer:

http://distributed.readthedocs.io/en/latest/quickstart.html

Either set up a dask-scheduler and dask-worker processes on your computer using the command line, or create a bare client below, which sets up a small cluster for you.  We recommend trying to use `dask-scheduler` and `dask-worker` directly first.

In [ ]:
# client = Client('localhost:8786')  # if you succeeded in setting up a dask-scheduler and dask-workers
# client = Client()  # if you just want to have Dask set things up for you

### Diagnostics

One of the main advantages of using the distributed scheduler is the diagnostics dashboards that should be hosted live at http://localhost:8787/status .  Visit that link and then run the computation again.

In [ ]:
%time s, d = dask.compute(spreads, days, get=client.get)  # This uses our "distributed" cluster

### Client takes over by default

Actually, we didn't need to add `get=client.get`.  The distributed scheduler takes over as the default scheduler by default when the Client is instantiated.

In [ ]:
%time s, d = dask.compute(spreads, days)  # This used to use threads by default, now it uses dask.distributed